Import Tensorflow and open a session.

In [2]:
%run 'EFL.py'

### Determinant
Calculate matrix determinant

In [4]:
W = tf.constant([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.matrix_determinant(W)
tf.Session().run(f)

9.0

Train by minimizing the ln(det(W)). TF knows how to do the gradient.

In [5]:
W = tf.Variable([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.log(tf.matrix_determinant(W))
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(f)
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train)
    print(sess.run(f))

2.06162
1.91632
1.75986
1.5904
1.4056


### Sparse Tensor
Sparse tensor must be converted to dense when passed to the determinant op.

In [337]:
W = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1., 2.], dense_shape=[2, 2])
W_dense = tf.sparse_tensor_to_dense(W)
f = tf.log(tf.matrix_determinant(W_dense))
sess.run(f)

0.69314718

## Entanglement
### Entanglement Regions

In [655]:
list(Region({1,2},4).equivalences())

[{2, 3}, {0, 1}, {1, 2}, {0, 3}]

In [654]:
Region({1,2},4).config()

array([ 1.000, -1.000, -1.000,  1.000])

In [651]:
list(RegionServer(8).consecutive())

[{0}, {0, 1}, {0, 1, 2}, {0, 1, 2, 3}]

In [653]:
list(RegionServer(6).representative())

[{0}, {0, 1}, {0, 2}, {0, 3}, {0, 1, 2}, {0, 1, 3}, {0, 2, 4}]

In [801]:
list(RegionServer(8).multiple(1,2))

[{1, 2, 3, 4, 6},
 {0, 1, 2},
 {0, 3, 4},
 {5},
 {0, 1, 4, 5},
 {0, 1, 3, 4, 5},
 {1, 2, 3, 4, 5, 6},
 {2, 3, 6}]

In [802]:
list(RegionServer(8).weighted())

[{3, 5, 6}, {3, 4, 5}, {4, 5}, {2}, {3, 4, 5}, {2, 3, 5, 6}, {0, 1}, {1}]

### Entanglement Entropy
Entanglement entropy of free fermion.
$$H=\sum_{i}(1+(-)^i m)\mathrm{i}\chi_{i}\chi_{i+1}$$

In [745]:
system = FreeFermion(mass=1.e-10, size=8)
system.S([0,1,2,3,4])

0.84523244468672853

### Square Lattice Adjacency Matices

In [297]:
%run 'Ising.py'
latt = SSLattice(width = 16, depth = 5, pattern = [-1,-1,1])
latt.print_structure(upto = 'cell')

depth =  5
Chain[0]
├ Cell[0, 0] IR
├ Cell[0, 1] IR
├ Cell[0, 2] IR
├ Cell[0, 3] IR
├ Cell[0, 4] IR
├ Cell[0, 5] IR
├ Cell[0, 6] IR
├ Cell[0, 7] IR
├ Cell[0, 8] IR
├ Cell[0, 9] IR
├ Cell[0, 10] IR
├ Cell[0, 11] IR
├ Cell[0, 12] IR
├ Cell[0, 13] IR
├ Cell[0, 14] IR
├ Cell[0, 15] IR
Chain[1]
├ Cell[1, 0] UV
├ Cell[1, 1] UV
├ Cell[1, 2] IR
├ Cell[1, 3] UV
├ Cell[1, 4] UV
├ Cell[1, 5] IR
├ Cell[1, 6] UV
├ Cell[1, 7] UV
├ Cell[1, 8] IR
├ Cell[1, 9] UV
├ Cell[1, 10] UV
├ Cell[1, 11] IR
├ Cell[1, 12] UV
├ Cell[1, 13] UV
├ Cell[1, 14] IR
├ Cell[1, 15] UV
├ Cell[1, 16] UV
├ Cell[1, 17] IR
├ Cell[1, 18] UV
├ Cell[1, 19] UV
├ Cell[1, 20] IR
├ Cell[1, 21] UV
├ Cell[1, 22] UV
├ Cell[1, 23] IR
Chain[2]
├ Cell[2, 0] UV
├ Cell[2, 1] UV
├ Cell[2, 2] IR
├ Cell[2, 3] UV
├ Cell[2, 4] UV
├ Cell[2, 5] IR
├ Cell[2, 6] UV
├ Cell[2, 7] UV
├ Cell[2, 8] IR
├ Cell[2, 9] UV
├ Cell[2, 10] UV
├ Cell[2, 11] IR
Chain[3]
├ Cell[3, 0] UV
├ Cell[3, 1] UV
├ Cell[3, 2] IR
├ Cell[3, 3] UV
├ Cell[3, 4] UV
├ Cell[3, 5] IR
Cha

## Performance
### Implementing LogDet
Define logdet and its gradient

In [1]:
import numpy as np
import tensorflow as tf
# from https://gist.github.com/harpone/3453185b41d8d985356cbe5e57d67342
# Define custom py_func which takes also a grad op as argument:
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    tf.RegisterGradient(rnd_name)(grad)  # see _MySquareGrad for grad example
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
# from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/linalg_grad.py
# Gradient for logdet
def logdet_grad(op, grad):
    a = op.inputs[0]
    a_adj_inv = tf.matrix_inverse(a, adjoint=True)
    out_shape = tf.concat([tf.shape(a)[:-2], [1, 1]], axis=0)
    return tf.reshape(grad, out_shape) * a_adj_inv
# define logdet by calling numpy.linalg.slogdet
def logdet(a, name = None):
    with tf.name_scope(name, 'LogDet', [a]) as name:
        res = py_func(lambda a: np.linalg.slogdet(a)[1], 
                      [a], 
                      tf.float64, 
                      name=name, 
                      grad=logdet_grad) # set the gradient
        return res

test

In [2]:
sess = tf.Session()
v = tf.Variable(np.array([[4.,0.,0.,2.],[2.,1.,3.,4.]]))
T = tf.constant(np.array([[[1.,0.],[0.,0.]],[[0.,1.],[0.,0.]],[[0.,0.],[1.,0.]],[[0.,0.],[0.,1.]]]))
x = tf.tensordot(v, T, axes = 1)
y = logdet(x)
g = tf.gradients(y, [v])
sess.run(tf.global_variables_initializer())
sess.run([x, y, g])

[array([[[ 4.,  0.],
         [ 0.,  2.]],
 
        [[ 2.,  1.],
         [ 3.,  4.]]]),
 array([ 2.07944154,  1.60943791]),
 [array([[ 0.25,  0.  ,  0.  ,  0.5 ],
         [ 0.8 , -0.6 , -0.2 ,  0.4 ]])]]

In [4]:
i = tf.constant(np.eye(500))
x = tf.Variable(np.array([10.]))
y = logdet(x*i)
dy = tf.gradients(y, [x])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([y, dy]))

[1151.2925464970251, [array([ 50.])]]


# Test Field

In [35]:
%run 'EFL.py'
machine = Machine(
    IsingModel(SSLattice(width=16,depth=100,pattern=[-1,-1,1])),
    FreeFermion(size=32,mass=1e-8,c=8),method='random')

In [45]:
for i in range(5):
    machine.train(20, learning_rate=0.02, beta1=0.9)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))

(0.00050477270583266945, array([ 0.659,  0.206,  0.206,  0.206]))
(0.0005532842579311308, array([ 0.663,  0.216,  0.216,  0.218]))
(0.0040939142210905969, array([ 0.664,  0.214,  0.214,  0.215]))
(0.00079832561194759818, array([ 0.662,  0.203,  0.203,  0.205]))
(0.0010922764639662301, array([ 0.664,  0.208,  0.208,  0.209]))


In [34]:
machine.save()

INFO:tensorflow:Saving parameters to ./machine/Ising(SSLatt(16,100,UUI))FF(32,0.00,8.0)random


In [40]:
machine.load()

INFO:tensorflow:Restoring parameters from ./machine/Ising(SSLatt(16,100,UUI))FF(32,0.00,8.0)random


In [16]:
list(machine.model.J.eval(machine.session))

[nan, nan, nan, nan]

In [1313]:
machine.session.run([machine.model.J])

[array([ 0.160,  0.114,  0.081,  0.051])]

In [1122]:
machine.session.run(machine.model.J.assign(
    [0.663,0.203,0.203,0.203,0.203]))

array([ 0.663,  0.203,  0.203,  0.203,  0.203])

In [304]:
machine.session.run(machine.model.J.assign(
        [0.26182317194354604,
         0.072700507415155574,
         0.075061925104706717,
         0.070486877299682602]))

array([ 0.262,  0.073,  0.075,  0.070])

In [1123]:
machine.session.run(machine.model.MSE, machine.data_server.fetch('random', batch=32))

0.0083049209485587978

In [1124]:
machine.session.run(machine.model.MSE, machine.data_server.fetch('consecutive'))

0.049255719390446075

In [6]:
machine.session.run(machine.step)

300

In [1166]:
datetime.datetime.now().strftime('%m%d%H%M%S')

'0609143842'

In [493]:
sess.run(b)

array([ 4.000,  2.000,  1.000,  1.000])

In [1173]:
''.join(str(x) for x in 'weighted')

'weighted'